In [1]:
here::i_am("atac/archR/chromvar/silicoChIP_ChromVAR.ipynb")

source(here::here("settings.R"))
source(here::here("utils.R"))

suppressPackageStartupMessages(library(ArchR))
suppressPackageStartupMessages(library(dplyr))

BPPARAM <- BiocParallel::bpparam()
BPPARAM$workers = 21

addArchRThreads(24)

here() starts at /rds/project/rds-SDzz0CATGms/users/bt392/09_Eomes_invitro_blood/code

Warning message:
“package ‘rtracklayer’ was built under R version 4.2.3”

                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | 

In [2]:
args = list()

# Metadata
args$metadata = file.path(io$basedir, 'results/rna_atac/clustering/metadata_celltype_annotated.txt.gz')

# Archr
args$archr_directory = file.path(io$basedir, 'processed/atac/archR')

# outdir
args$outdir = file.path(io$basedir, 'results/rna_atac/differential/Accessibility/')
dir.create(args$outdir, recursive=TRUE, showWarnings =FALSE)

In [3]:
# Load meta
meta = fread(args$metadata)

#### Load Data

In [4]:
ArchRProject <- loadArchRProject(args$archr_directory)[meta$cell]

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .____

In [5]:
getAvailableMatrices(ArchRProject)

[1] "DeviationMatrix_CISBP"   "DeviationMatrix_JASPAR" 
 [3] "GeneScoreMatrix_TSS"     "GeneScoreMatrix_distal" 
 [5] "MarkerPeaksMatrix"       "MotifMatrix"            
 [7] "PeakMatrix"              "PeakMatrix_atlas"       
 [9] "TileMatrix"              "silicoChIP_CISBPMatrix" 
[11] "silicoChIP_CISBP_v2"     "silicoChIP_JASPARMatrix"
[13] "silicoChIP_JASPAR_v2"

#### Cisbp in silico chip

In [29]:
io$multiome = '/rds/project/rds-SDzz0CATGms/users/bt392/09_Eomes_invitro_blood/results/multiome_atlas/'
cisbp_silico_chip = readRDS(file.path(io$multiome, 'vitro_peaks_silicoChIP_cisbp.rds'))

In [30]:
assay(cisbp_silico_chip, 'matches') = assay(cisbp_silico_chip, 'VirtualChipScores') > 0.20

In [31]:
assay(cisbp_silico_chip, 'motifScores') = NULL
assay(cisbp_silico_chip, 'motifMatches') = NULL
assay(cisbp_silico_chip, 'motifCounts') = NULL
assay(cisbp_silico_chip, 'VirtualChipScores') = NULL

In [32]:
# Keep only motifs with more than 40 peaks 
tmp = colSums(assay(cisbp_silico_chip, 'matches') )
keep = names(tmp[tmp > 40])
cisbp_silico_chip = cisbp_silico_chip[,keep]

In [33]:
addArchRThreads(12)
ArchRProject <- addDeviationsMatrix(
      ArchRProj = ArchRProject, 
      matches = cisbp_silico_chip,
      matrixName = 'silicoChIP_CISBP_v2',
      force = TRUE
    )

Setting default number of Parallel threads to 12.

Using Previous Background Peaks!

ArchR logging to : ArchRLogs/ArchR-addDeviationsMatrix-26a9cd529eaae6-Date-2024-04-26_Time-17-46-19.log
If there is an issue, please report to github with logFile!



class: RangedSummarizedExperiment 
dim: 234908 534 
metadata(0):
assays(1): matches
rownames(234908): chr1:3003470-3004070 chr1:3035598-3036198 ...
  chrX:169925416-169926016 chrX:169937236-169937836
rowData names(13): score replicateScoreQuantile ... idx N
colnames(534): AHCTF1 AHRR ... ZSCAN10 ZSCAN20
colData names(1): name


2024-04-26 17:46:31 : Batch Execution w/ safelapply!, 0 mins elapsed.

###########
2024-04-26 18:21:55 : Completed Computing Deviations!, 35.593 mins elapsed.
###########

ArchR logging successful to : ArchRLogs/ArchR-addDeviationsMatrix-26a9cd529eaae6-Date-2024-04-26_Time-17-46-19.log



In [34]:
addArchRThreads(5)
cisbp_silico_chip = getMatrixFromProject(ArchRProject, useMatrix = 'MotifMatrix')
saveRDS(cisbp_silico_chip, file.path(args$archr_directory, '/Matrices/cisbp_silico_chip.rds'))

Setting default number of Parallel threads to 5.

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-26a9cd62e06ea-Date-2024-04-26_Time-18-21-56.log
If there is an issue, please report to github with logFile!

2024-04-26 18:22:06 : Organizing colData, 0.164 mins elapsed.

2024-04-26 18:22:07 : Organizing rowData, 0.175 mins elapsed.

2024-04-26 18:22:07 : Organizing rowRanges, 0.175 mins elapsed.

2024-04-26 18:22:07 : Organizing Assays (1 of 2), 0.175 mins elapsed.

2024-04-26 18:22:11 : Organizing Assays (2 of 2), 0.247 mins elapsed.

2024-04-26 18:22:14 : Constructing SummarizedExperiment, 0.29 mins elapsed.

2024-04-26 18:22:15 : Finished Matrix Creation, 0.304 mins elapsed.



#### Jaspar2020 (human) in silico chip

In [ ]:
io$multiome = '/rds/project/rds-SDzz0CATGms/users/bt392/09_Eomes_invitro_blood/results/multiome_atlas/'
jaspar_silico_chip = readRDS(file.path(io$multiome, 'vitro_peaks_silicoChIP_jaspar2020.rds'))

In [ ]:
assay(jaspar_silico_chip, 'matches') = assay(jaspar_silico_chip, 'VirtualChipScores') > 0.20

In [ ]:
assay(jaspar_silico_chip, 'motifScores') = NULL
assay(jaspar_silico_chip, 'motifMatches') = NULL
assay(jaspar_silico_chip, 'motifCounts') = NULL
assay(jaspar_silico_chip, 'VirtualChipScores') = NULL

In [ ]:
# Keep only motifs with more than 40 peaks 
tmp = colSums(assay(jaspar_silico_chip, 'matches') )
keep = names(tmp[tmp > 40])
jaspar_silico_chip = jaspar_silico_chip[,keep]

In [ ]:
addArchRThreads(12)
ArchRProject <- addDeviationsMatrix(
      ArchRProj = ArchRProject, 
      matches = jaspar_silico_chip,
      matrixName = 'silicoChIP_JASPAR_v2',
      force = TRUE
    )

In [ ]:
addArchRThreads(5)
jaspar_silico_chip = getMatrixFromProject(ArchRProject, useMatrix = 'silicoChIP_JASPAR_v2')
saveRDS(jaspar_silico_chip, file.path(args$archr_directory, '/Matrices/jaspar_silico_chip.rds'))

# Old method (not ideal)

### Add silico ChIP Peak annotation

In [40]:
test = ArchRProject

In [41]:
# make dictionary of bed files from in silico ChIP
files = list.files('/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/results/rna_atac/virtual_chipseq/pseudobulk/JASPAR', pattern='.bed.gz')
files_dict = files
files_dict = paste0('/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/results/rna_atac/virtual_chipseq/pseudobulk/JASPAR/', files)
names(files_dict) = str_split(files, '\\.') %>% map_chr(1)

# keep only the top 1000 predictions
mclapply(names(files_dict), function(x){
    tmp = fread(files_dict[[x]])[order(-V4)][V4>0.25] # only select peaks with >0.25 & keep only top 1000
    tmp2 = head(tmp, 1000)
    if(nrow(tmp2)>25){
        fwrite(tmp2, gsub('.bed.gz', '.top_bed.gz', files_dict[[x]]), col.names=F)
    }}, mc.cores=12)

# make dictionary of bed files from in silico ChIP
files = list.files('/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/results/rna_atac/virtual_chipseq/pseudobulk/JASPAR', pattern='.top_bed.gz')
files_dict = files
files_dict = paste0('/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/results/rna_atac/virtual_chipseq/pseudobulk/JASPAR/', files)
names(files_dict) = str_split(files, '\\.') %>% map_chr(1)

# Only testing now with motifs keep to keep chromvar quick
test <- addPeakAnnotations(ArchRProj = test, regions = files_dict, name = "silicoChIP_JASPAR", force=T)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

[[72]]
NULL

[[73]]
NULL

[[74]]
NULL

[[75]]
NULL

[[76]]
NULL

[[77]]
NULL

[[78]]
NULL

[[79]]
NULL

[[80]]
NULL

[[81]]
NULL

[[82]]
NULL

[[83]]
NULL

[[84]]
NULL

[[85]]
NULL

[[86]]
NULL

[[87]]
NULL

[[88]]
NULL

[[89]]
NULL

[[90]]
NULL

[[91]]
NULL

[[92]]
NULL

[[93]]
NULL

[[94]]
NULL

[[95]]
NULL

[[96]]
NULL

[[97]]
NULL

[[98]]
NULL

[[99]]
NULL

[[100]]
NULL

[[101]]
NULL

[[102]]
NULL

[[103]]
NULL

[[104]]
NULL

[[105]]
NULL

[[106]]
NULL

[[107]]
NULL

[[108]]
NULL

[[109]]
NULL

[[110]]
NULL

[[111]]
NULL

[[112]]
NULL

[[113]]
NULL

[[114]]
NULL

[[115]]
NULL

[[116]]
NULL

[[117]]
NULL

[[118]]
NULL

[[119]]
NULL

[[120]]
NULL

[[121]]
NULL

[[122]]
NULL

[[123]]
NULL

[[124]]
NULL

[[125]]
NULL

[[126]]
NULL

[[127]]
NULL

[[128]]
NULL

[[129]]
NULL

[[130]]
NULL

[[131]]
NULL

[[132]]
NULL

[[133]]
NULL

[[134]]
NULL

[[135]]
NULL

[[136]]
NULL

[[137]]
NULL

[[138]]
NULL

[[139]]
NULL

[[140]]
NULL

[[141]]
NULL

[[142]]
NULL

[[143]]
NULL

[[144]]
NULL

[[145]]
NULL

[[146]]
NULL

[[147]]
NULL

[[148]]
NULL

[[149]]
NULL

[[150]]
NULL

[[151]]
NULL

[[152]]
NULL

[[153]]
NULL

[[154]]
NULL

[[155]]
NULL

[[156]]
NULL

[[157]]
NULL

[[158]]
NULL

[[159]]
NULL

[[160]]
NULL

[[161]]
NULL

[[162]]
NULL

[[163]]
NULL

[[164]]
NULL

[[165]]
NULL

[[166]]
NULL

[[167]]
NULL

[[168]]
NULL

[[169]]
NULL

[[170]]
NULL

[[171]]
NULL

[[172]]
NULL

[[173]]
NULL

[[174]]
NULL

[[175]]
NULL

[[176]]
NULL

[[177]]
NULL

[[178]]
NULL

[[179]]
NULL

[[180]]
NULL

[[181]]
NULL

[[182]]
NULL

[[183]]
NULL

[[184]]
NULL

[[185]]
NULL

[[186]]
NULL

[[187]]
NULL

[[188]]
NULL

[[189]]
NULL

[[190]]
NULL

[[191]]
NULL

[[192]]
NULL

[[193]]
NULL

[[194]]
NULL

[[195]]
NULL

[[196]]
NULL

[[197]]
NULL

[[198]]
NULL

[[199]]
NULL

[[200]]
NULL

[[201]]
NULL

[[202]]
NULL

[[203]]
NULL

[[204]]
NULL

[[205]]
NULL

[[206]]
NULL

[[207]]
NULL

[[208]]
NULL

[[209]]
NULL

[[210]]
NULL

[[211]]
NULL

[[212]]
NULL

[[213]]
NULL

[[214]]
NULL

[[215]]
NULL

[[216]]
NULL

[[217]]
NULL

[[218]]
NULL

[[219]]
NULL

[[220]]
NULL

[[221]]
NULL

[[222]]
NULL

[[223]]
NULL

[[224]]
NULL

[[225]]
NULL

[[226]]
NULL

[[227]]
NULL

[[228]]
NULL

[[229]]
NULL

[[230]]
NULL

[[231]]
NULL

[[232]]
NULL

[[233]]
NULL

[[234]]
NULL

[[235]]
NULL

[[236]]
NULL

[[237]]
NULL

[[238]]
NULL

[[239]]
NULL

[[240]]
NULL

[[241]]
NULL

[[242]]
NULL

[[243]]
NULL

[[244]]
NULL

[[245]]
NULL

[[246]]
NULL

[[247]]
NULL

[[248]]
NULL

[[249]]
NULL

[[250]]
NULL

[[251]]
NULL

[[252]]
NULL

[[253]]
NULL

[[254]]
NULL

[[255]]
NULL

[[256]]
NULL

[[257]]
NULL

[[258]]
NULL

[[259]]
NULL

[[260]]
NULL

[[261]]
NULL

[[262]]
NULL

[[263]]
NULL

[[264]]
NULL

[[265]]
NULL

[[266]]
NULL

[[267]]
NULL

[[268]]
NULL

[[269]]
NULL

[[270]]
NULL

[[271]]
NULL

[[272]]
NULL

[[273]]
NULL

[[274]]
NULL

[[275]]
NULL

[[276]]
NULL

[[277]]
NULL

[[278]]
NULL

[[279]]
NULL

[[280]]
NULL

[[281]]
NULL

[[282]]
NULL

[[283]]
NULL

[[284]]
NULL

[[285]]
NULL

[[286]]
NULL

[[287]]
NULL

[[288]]
NULL

[[289]]
NULL

[[290]]
NULL

[[291]]
NULL

[[292]]
NULL

[[293]]
NULL

[[294]

ArchR logging to : ArchRLogs/ArchR-addPeakAnnotations-129bff13f64da8-Date-2023-05-27_Time-16-19-35.log
If there is an issue, please report to github with logFile!

2023-05-27 16:19:54 : Creating Peak Overlap Matrix, 0.306 mins elapsed.

2023-05-27 16:19:54 : All Regions Overlap at least 1 peak!, 0.309 mins elapsed.



In [42]:
# junk <- dir(path="/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/results/rna_atac/virtual_chipseq/pseudobulk/JASPAR/",  pattern=".top_bed.gz") # ?dir
# file.remove(paste0("/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/results/rna_atac/virtual_chipseq/pseudobulk/JASPAR/", junk)) # ?file.remove

In [43]:
# make dictionary of bed files from in silico ChIP
files = list.files('/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/results/rna_atac/virtual_chipseq/pseudobulk/CISBP', pattern='.bed.gz')
files_dict = files
files_dict = paste0('/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/results/rna_atac/virtual_chipseq/pseudobulk/CISBP/', files)
names(files_dict) = str_split(files, '\\.') %>% map_chr(1)

# keep only the top 1000 predictions
mclapply(names(files_dict), function(x){
    tmp = fread(files_dict[[x]])[order(-V4)][V4>0.25]
    tmp2 = head(tmp, 1000)
    if(nrow(tmp2)>25){
        fwrite(tmp2, gsub('.bed.gz', '.top_bed.gz', files_dict[[x]]), col.names=F)
    }
}, mc.cores=12)

# make dictionary of bed files from in silico ChIP
files = list.files('/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/results/rna_atac/virtual_chipseq/pseudobulk/CISBP', pattern='.top_bed.gz')
files_dict = files
files_dict = paste0('/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/results/rna_atac/virtual_chipseq/pseudobulk/CISBP/', files)
names(files_dict) = str_split(files, '\\.') %>% map_chr(1)

# Only testing now with motifs keep to keep chromvar quick
test <- addPeakAnnotations(ArchRProj = test, regions = files_dict, name = "silicoChIP_CISBP", force=T)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

[[72]]
NULL

[[73]]
NULL

[[74]]
NULL

[[75]]
NULL

[[76]]
NULL

[[77]]
NULL

[[78]]
NULL

[[79]]
NULL

[[80]]
NULL

[[81]]
NULL

[[82]]
NULL

[[83]]
NULL

[[84]]
NULL

[[85]]
NULL

[[86]]
NULL

[[87]]
NULL

[[88]]
NULL

[[89]]
NULL

[[90]]
NULL

[[91]]
NULL

[[92]]
NULL

[[93]]
NULL

[[94]]
NULL

[[95]]
NULL

[[96]]
NULL

[[97]]
NULL

[[98]]
NULL

[[99]]
NULL

[[100]]
NULL

[[101]]
NULL

[[102]]
NULL

[[103]]
NULL

[[104]]
NULL

[[105]]
NULL

[[106]]
NULL

[[107]]
NULL

[[108]]
NULL

[[109]]
NULL

[[110]]
NULL

[[111]]
NULL

[[112]]
NULL

[[113]]
NULL

[[114]]
NULL

[[115]]
NULL

[[116]]
NULL

[[117]]
NULL

[[118]]
NULL

[[119]]
NULL

[[120]]
NULL

[[121]]
NULL

[[122]]
NULL

[[123]]
NULL

[[124]]
NULL

[[125]]
NULL

[[126]]
NULL

[[127]]
NULL

[[128]]
NULL

[[129]]
NULL

[[130]]
NULL

[[131]]
NULL

[[132]]
NULL

[[133]]
NULL

[[134]]
NULL

[[135]]
NULL

[[136]]
NULL

[[137]]
NULL

[[138]]
NULL

[[139]]
NULL

[[140]]
NULL

[[141]]
NULL

[[142]]
NULL

[[143]]
NULL

[[144]]
NULL

[[145]]
NULL

[[146]]
NULL

[[147]]
NULL

[[148]]
NULL

[[149]]
NULL

[[150]]
NULL

[[151]]
NULL

[[152]]
NULL

[[153]]
NULL

[[154]]
NULL

[[155]]
NULL

[[156]]
NULL

[[157]]
NULL

[[158]]
NULL

[[159]]
NULL

[[160]]
NULL

[[161]]
NULL

[[162]]
NULL

[[163]]
NULL

[[164]]
NULL

[[165]]
NULL

[[166]]
NULL

[[167]]
NULL

[[168]]
NULL

[[169]]
NULL

[[170]]
NULL

[[171]]
NULL

[[172]]
NULL

[[173]]
NULL

[[174]]
NULL

[[175]]
NULL

[[176]]
NULL

[[177]]
NULL

[[178]]
NULL

[[179]]
NULL

[[180]]
NULL

[[181]]
NULL

[[182]]
NULL

[[183]]
NULL

[[184]]
NULL

[[185]]
NULL

[[186]]
NULL

[[187]]
NULL

[[188]]
NULL

[[189]]
NULL

[[190]]
NULL

[[191]]
NULL

[[192]]
NULL

[[193]]
NULL

[[194]]
NULL

[[195]]
NULL

[[196]]
NULL

[[197]]
NULL

[[198]]
NULL

[[199]]
NULL

[[200]]
NULL

[[201]]
NULL

[[202]]
NULL

[[203]]
NULL

[[204]]
NULL

[[205]]
NULL

[[206]]
NULL

[[207]]
NULL

[[208]]
NULL

[[209]]
NULL

[[210]]
NULL

[[211]]
NULL

[[212]]
NULL

[[213]]
NULL

[[214]]
NULL

[[215]]
NULL

[[216]]
NULL

[[217]]
NULL

[[218]]
NULL

[[219]]
NULL

[[220]]
NULL

[[221]]
NULL

[[222]]
NULL

[[223]]
NULL

[[224]]
NULL

[[225]]
NULL

[[226]]
NULL

[[227]]
NULL

[[228]]
NULL

[[229]]
NULL

[[230]]
NULL

[[231]]
NULL

[[232]]
NULL

[[233]]
NULL

[[234]]
NULL

[[235]]
NULL

[[236]]
NULL

[[237]]
NULL

[[238]]
NULL

[[239]]
NULL

[[240]]
NULL

[[241]]
NULL

[[242]]
NULL

[[243]]
NULL

[[244]]
NULL

[[245]]
NULL

[[246]]
NULL

[[247]]
NULL

[[248]]
NULL

[[249]]
NULL

[[250]]
NULL

[[251]]
NULL

[[252]]
NULL

[[253]]
NULL

[[254]]
NULL

[[255]]
NULL

[[256]]
NULL

[[257]]
NULL

[[258]]
NULL

[[259]]
NULL

[[260]]
NULL

[[261]]
NULL

[[262]]
NULL

[[263]]
NULL

[[264]]
NULL

[[265]]
NULL

[[266]]
NULL

[[267]]
NULL

[[268]]
NULL

[[269]]
NULL

[[270]]
NULL

[[271]]
NULL

[[272]]
NULL

[[273]]
NULL

[[274]]
NULL

[[275]]
NULL

[[276]]
NULL

[[277]]
NULL

[[278]]
NULL

[[279]]
NULL

[[280]]
NULL

[[281]]
NULL

[[282]]
NULL

[[283]]
NULL

[[284]]
NULL

[[285]]
NULL

[[286]]
NULL

[[287]]
NULL

[[288]]
NULL

[[289]]
NULL

[[290]]
NULL

[[291]]
NULL

[[292]]
NULL

[[293]]
NULL

[[294]

ArchR logging to : ArchRLogs/ArchR-addPeakAnnotations-129bff3ed81649-Date-2023-05-27_Time-16-20-00.log
If there is an issue, please report to github with logFile!

2023-05-27 16:20:28 : Creating Peak Overlap Matrix, 0.456 mins elapsed.

2023-05-27 16:20:28 : All Regions Overlap at least 1 peak!, 0.459 mins elapsed.



In [44]:
test@peakAnnotation

List of length 4
names(4): Motif_cisbp Motif_JASPAR2020 silicoChIP_JASPAR silicoChIP_CISBP

### Add ChromVAR for RNA_ATAC cells

In [45]:
test <- addDeviationsMatrix(
      ArchRProj = test, 
      peakAnnotation = "silicoChIP_CISBP",
      force = TRUE
    )

Using Previous Background Peaks!

ArchR logging to : ArchRLogs/ArchR-addDeviationsMatrix-129bff34ee06e0-Date-2023-05-27_Time-16-23-36.log
If there is an issue, please report to github with logFile!



NULL


as(<lgCMatrix>, "dgCMatrix") is deprecated since Matrix 1.5-0; do as(., "dMatrix") instead

2023-05-27 16:24:15 : Batch Execution w/ safelapply!, 0 mins elapsed.

###########
2023-05-27 16:38:32 : Completed Computing Deviations!, 14.945 mins elapsed.
###########

ArchR logging successful to : ArchRLogs/ArchR-addDeviationsMatrix-129bff34ee06e0-Date-2023-05-27_Time-16-23-36.log



In [46]:
test <- addDeviationsMatrix(
      ArchRProj = test, 
      peakAnnotation = "silicoChIP_JASPAR",
      force = TRUE
    )

Using Previous Background Peaks!

ArchR logging to : ArchRLogs/ArchR-addDeviationsMatrix-129bffcd49d1f-Date-2023-05-27_Time-16-46-09.log
If there is an issue, please report to github with logFile!



NULL


2023-05-27 16:47:38 : Batch Execution w/ safelapply!, 0 mins elapsed.

###########
2023-05-27 16:57:27 : Completed Computing Deviations!, 11.307 mins elapsed.
###########

ArchR logging successful to : ArchRLogs/ArchR-addDeviationsMatrix-129bffcd49d1f-Date-2023-05-27_Time-16-46-09.log

